In [1]:
# import nltk
# from nltk.corpus import reuters
# from nltk import bigrams, trigrams
import numpy as np 
# import pandas as pd
import keras
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential 
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
import re
# from collections import Counter, defaultdict

In [31]:
text = open('shakespeare.txt', 'r').read().lower()[:60000]

In [32]:
def create_sequences(text):
    length = 30
    sequences = []
    for i in range(length, len(text)):
        seq = text[i-length:i+1]
        sequences.append(seq)
    print("Total Sequences: {}".format(len(sequences)))
    return sequences
sequences = create_sequences(text)
# print(sequences[0])
chars = sorted(list(set(text)))
mapping = dict((c,i) for i,c in enumerate(chars))
def encode_seq(seq):
    sequences = []
    for line in seq: 
        encoded_seq = [mapping[char] for char in line]
        sequences.append(encoded_seq)
#     print(encode_seq)
    return sequences
sequences = encode_seq(sequences)

Total Sequences: 59970


In [33]:
from sklearn.model_selection import train_test_split

vocab = len(mapping)
sequences = np.asarray(sequences)

X, y= sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab)

train_x, val_x, train_y, val_y = train_test_split(X,y,test_size = 0.1, random_state = 49) # for reproductibility

In [34]:
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 30, 50)            1800      
_________________________________________________________________
gru_6 (GRU)                  (None, 150)               90900     
_________________________________________________________________
dense_6 (Dense)              (None, 36)                5436      
Total params: 98,136
Trainable params: 98,136
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
1687/1687 [==============================] - 184s 109ms/step - loss: 2.4038 - acc: 0.3045 - val_loss: 2.1732 - val_acc: 0.3573
Epoch 2/100
1687/1687 [==============================] - 182s 108ms/step - loss: 2.0811 - acc: 0.3865 - val_loss: 2.0037 - val_acc: 0.4080
Epoch 3/100
1687/1687 [==============================] - 184s 109ms/step - loss: 1.9464 - acc: 0.

In [39]:
# model.save('/content/drive/MyDrive/NLP')
reconstructed_model = keras.models.load_model("/NLP")
# reconstructed_model.summary()


In [2]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

In [4]:
inp = 'a'
reconstructed_model = keras.models.load_model("NLP")
print(generate_seq(reconstructed_model, mapping, 30, inp.lower(),30)) 

NameError: name 'mapping' is not defined

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cd drive

In [ ]:
model.save('/content/drive/MyDrive')